In [1]:
import numpy as np
np.__version__


'1.26.4'

In [2]:
import scipy as sp
sp.__version__

'1.12.0'

In [10]:
class ExtendedNDArray(np.ndarray):
    def __new__(cls, input_array, *args, **kwargs):
        # Input array is an already formed ndarray instance
        # We first cast to be our class type
        obj = np.asarray(input_array).view(cls)
        return obj

    def __array_ufunc__(self, ufunc, method, *inputs, out=None, **kwargs):
        print(f"{method=}")
        cls = self.__class__
        args = []
        in_no = []
        for i, input_ in enumerate(inputs):
            if isinstance(input_, cls):
                in_no.append(i)
                args.append(input_.view(np.ndarray))
            else:
                args.append(input_)

        outputs = out
        out_no = []
        if outputs:
            out_args = []
            for j, output in enumerate(outputs):
                if isinstance(output, cls):
                    out_no.append(j)
                    out_args.append(output.view(np.ndarray))
                else:
                    out_args.append(output)
            kwargs['out'] = tuple(out_args)
        else:
            outputs = (None,) * ufunc.nout

        results = super().__array_ufunc__(ufunc, method, *args, **kwargs)
        if results is NotImplemented:
            return NotImplemented


        if method == 'at':
            return

        if method == "__call__":
            def cast_back_to_class(result):
                # do not cast
                return np.asarray(result).view(cls)
        else:
            def cast_back_to_class(result):
                # do not cast
                return result

        if ufunc.nout == 1:
            results = (results,)

        results = tuple((cast_back_to_class(result)
                         if output is None else output)
                        for result, output in zip(results, outputs))

        return results[0] if len(results) == 1 else results

class Image(ExtendedNDArray):
    def __new__(cls, *args, **kwargs):
        print("Image new")
        return super().__new__(cls, *args, **kwargs)
    def __array_finalize__ (self, obj):
        print(f"finalize {obj}")
        if obj is None: return
        print(f"finalize1 {obj}")
        
        if len(self.shape) == 2:
            print(f"finalize2 {obj}")
            return obj
        else:
            print(f"finalize3 {obj}")
            return 3
    @property
    def width(self):
        return self.shape[1]
    @property
    def height(self):
        return self.shape[0]

In [11]:
image = Image(np.arange(20).reshape((4, 5)))
arr = np.arange(20).reshape((4, 5))

Image new
finalize [[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]]
finalize1 [[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]]
finalize2 [[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]]


In [12]:
def get_view_name(a):
    if '(' not in repr(a):
        return None
    return repr(a).partition('(')[0]

In [13]:
image.reshape((-1,))

finalize [[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]]
finalize1 [[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]]
finalize3 [[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]]


Image([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

In [16]:
image[:2]

finalize [[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]]
finalize1 [[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]]
finalize2 [[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]]


Image([[0, 1, 2, 3, 4],
       [5, 6, 7, 8, 9]])

In [8]:
should_be_image = [
    image+1,
    image*image,
    image.astype(np.uint8),
    np.fmax(image, image),
    np.sin(image),
    np.sin(image),
    np.argsort(image),
    sp.special.i0(image),
    np.clip(image, 3, 5),
    np.pad(image, (2, 3), 'constant', constant_values=(4, 6)),
    image.reshape((-1,)),  # reshape is a view change
]

should_be_array = [
    np.median(image, axis=1),
    image.cumsum(),
    image.sum(0),
    image.sum(1),
]
should_be_none = [
    np.amin(image),
    np.min(image),
    np.nanprod(image),
    image.sum(),
]


method='__call__'
finalize [[ 1  2  3  4  5]
 [ 6  7  8  9 10]
 [11 12 13 14 15]
 [16 17 18 19 20]]
finalize1 [[ 1  2  3  4  5]
 [ 6  7  8  9 10]
 [11 12 13 14 15]
 [16 17 18 19 20]]
finalize2 [[ 1  2  3  4  5]
 [ 6  7  8  9 10]
 [11 12 13 14 15]
 [16 17 18 19 20]]
method='__call__'
finalize [[  0   1   4   9  16]
 [ 25  36  49  64  81]
 [100 121 144 169 196]
 [225 256 289 324 361]]
finalize1 [[  0   1   4   9  16]
 [ 25  36  49  64  81]
 [100 121 144 169 196]
 [225 256 289 324 361]]
finalize2 [[  0   1   4   9  16]
 [ 25  36  49  64  81]
 [100 121 144 169 196]
 [225 256 289 324 361]]
finalize [[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]]
finalize1 [[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]]
finalize2 [[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]]
method='__call__'
finalize [[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]]
finalize1 [[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 

In [9]:
for a in should_be_image:
    assert get_view_name(a) == 'Image'
    a.width
    a.height
for a in should_be_array:
    assert get_view_name(a) == 'array'
for a in should_be_none:
    assert get_view_name(a) is None

AssertionError: 